In [1]:
import pyfesom2 as pf
import numpy as np
import matplotlib.pylab as plt
import matplotlib.cm as cm
import xarray as xr
import os
from dask.distributed import LocalCluster, Client

In [2]:
mesh = pf.load_mesh('/work/ollie/clidyn/FESOM2/meshes/farc/')
datapath = "/work/ollie/nkolduno/farc_2.1_default_TKE/"

/work/ollie/clidyn/FESOM2/meshes/farc/pickle_mesh_py3_fesom2
The usepickle == True)
The pickle file for FESOM2 exists.
The mesh will be loaded from /work/ollie/clidyn/FESOM2/meshes/farc/pickle_mesh_py3_fesom2


In [3]:
cluster = LocalCluster(ip="0.0.0.0",
                       silence_logs=50,
                       n_workers=15,
                       threads_per_worker=1,
                       memory_limit='7G')
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://10.10.7.153:8787/status,
Dashboard: http://10.10.7.153:8787/status,Workers: 15
Total threads: 15,Total memory: 97.79 GiB
Status: running,Using processes: True
Comm: tcp://10.10.7.153:44569,Workers: 15
Dashboard: http://10.10.7.153:8787/status,Total threads: 15
Started: Just now,Total memory: 97.79 GiB
Comm: tcp://10.10.7.153:38885,Total threads: 1
Dashboard: http://10.10.7.153:34785/status,Memory: 6.52 GiB
Nanny: tcp://10.10.7.153:40759,


In [4]:
data = xr.open_mfdataset(os.path.join(datapath,"temp.fesom.*.nc"), chunks={'time':12,'nz1':5})

In [10]:
data

,Array,Chunk
Bytes,83.16 GiB,146.12 MiB
Shape,"(744, 638387, 47)","(12, 638387, 5)"
Count,1302 Tasks,620 Chunks
Type,float32,numpy.ndarray


In [6]:
a = (data.temp * data.temp).mean(dim='time')

In [7]:
a

<xarray.DataArray 'temp' (nod2: 638387, nz1: 47)>
dask.array<mean_agg-aggregate, shape=(638387, 47), dtype=float32, chunksize=(638387, 5), chunktype=numpy.ndarray>
Coordinates:
  * nz1      (nz1) float64 2.5 7.5 15.0 25.0 ... 5.525e+03 5.825e+03 6.125e+03
Dimensions without coordinates: nod2

In [8]:
%%time
a.compute()

CPU times: user 10.9 s, sys: 1.51 s, total: 12.5 s
Wall time: 37.2 s


<xarray.DataArray 'temp' (nod2: 638387, nz1: 47)>
array([[2.7290673, 2.7277536, 0.       , ..., 0.       , 0.       ,
        0.       ],
       [2.677601 , 2.6703362, 2.654298 , ..., 0.       , 0.       ,
        0.       ],
       [2.7493663, 2.739532 , 2.726278 , ..., 0.       , 0.       ,
        0.       ],
       ...,
       [2.4451852, 2.438488 , 2.4293783, ..., 0.       , 0.       ,
        0.       ],
       [2.532501 , 2.5233908, 2.510712 , ..., 0.       , 0.       ,
        0.       ],
       [2.5677292, 2.5579984, 2.5442228, ..., 0.       , 0.       ,
        0.       ]], dtype=float32)
Coordinates:
  * nz1      (nz1) float64 2.5 7.5 15.0 25.0 ... 5.525e+03 5.825e+03 6.125e+03
Dimensions without coordinates: nod2